<a href="https://colab.research.google.com/github/ekosgei/f1-pitstop-strategy-data-wrangling/blob/main/f1_pitstop_strategy_data_wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastf1 pandas numpy matplotlib

In [ ]:
import fastf1
import pandas as pd
race = fastf1.get_session(2025, 'Mexico Grand Prix', 'R')
race.load()

req         WARNING 	DEFAULT CACHE ENABLED! (198.27 MB) /root/.cache/fastf1
core           INFO 	Loading data for Mexico City Grand Prix - Race [v3.7.0]
INFO:fastf1.fastf1.core:Loading data for Mexico City Grand Prix - Race [v3.7.0]
req            INFO 	Using cached data for session_info
INFO:fastf1.fastf1.req:Using cached data for session_info
req            INFO 	Using cached data for driver_info
INFO:fastf1.fastf1.req:Using cached data for driver_info
DEBUG:fastf1.ergast:Failed to parse timestamp '-1:55:35.558' in Ergastresponse.
req            INFO 	Using cached data for session_status_data
INFO:fastf1.fastf1.req:Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
INFO:fastf1.fastf1.req:Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
INFO:fastf1.fastf1.req:Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
INFO:fastf1.fastf1.req:Using cached 

In [ ]:
laps = race.laps
##pits = laps[['Driver','PitInLap','PitOutLap','TyreCompound','Stint']].dropna()
pit_stops = laps.pick_box_laps(which="in")
#drop formation laps
pit_stops = pit_stops.dropna(subset=['LapTime'])
pit_stops = pit_stops[['Team','Driver','LapNumber','Position', 'Stint']]

lap_positions = laps[['Driver', 'LapNumber', 'Position', 'Stint']]
#finding lap position before pitting
pit_stops['Previous_Lap'] = pit_stops['LapNumber'] - 1
pit_stops = pit_stops.merge(
    lap_positions,
    left_on= ['Driver', 'Previous_Lap'],
    right_on = ['Driver', 'LapNumber'],
    how='left',
    suffixes=('_pit', '_before')
)
pit_stops = pit_stops.drop(columns=['LapNumber_before','Stint_before'])

pit_stops = pit_stops.rename(columns={
    'LapNumber_pit': 'PitLap',
    'Position_pit': 'PositionAtPit',
    'Previous_Lap': 'LapBeforePit',
    'Position_before': 'PositionBeforePit',
    'Stint_pit': 'Stint'
})

pit_stops

,Team,Driver,PitLap,PositionAtPit,Stint,LapBeforePit,PositionBeforePit
0,Red Bull Racing,VER,37.0,2.0,1.0,36.0,2.0
1,Alpine,GAS,33.0,18.0,1.0,32.0,17.0
2,Mercedes,ANT,22.0,8.0,1.0,21.0,6.0
3,Mercedes,ANT,47.0,6.0,2.0,46.0,5.0
4,Aston Martin,ALO,20.0,15.0,1.0,19.0,13.0
5,Aston Martin,ALO,34.0,16.0,2.0,33.0,15.0
6,Ferrari,LEC,31.0,2.0,1.0,30.0,2.0
7,Aston Martin,STR,26.0,12.0,1.0,25.0,11.0
8,Aston Martin,STR,43.0,11.0,2.0,42.0,11.0
9,Red Bull Racing,TSU,36.0,9.0,1.0,35.0,9.0


In [ ]:
# Get Rival Driver
pit_stops['Rival_Position'] = pit_stops['PositionBeforePit'] - 1
pit_stops = pit_stops.merge(
    lap_positions,
    left_on=['LapBeforePit', 'Rival_Position'],
    right_on=['LapNumber', 'Position'],
    how='left',
    suffixes=('', '_rival')
)
pit_stops = pit_stops.drop(columns=['LapNumber','Position'])


pit_stops.groupby(['Driver', 'Stint']).size().describe()

pit_stops.head()

,Team,Driver,PitLap,PositionAtPit,Stint,LapBeforePit,PositionBeforePit,Rival_Position,Driver_rival,Stint_rival
0,Red Bull Racing,VER,37.0,2.0,1.0,36.0,2.0,1.0,NOR,2.0
1,Alpine,GAS,33.0,18.0,1.0,32.0,17.0,16.0,STR,2.0
2,Mercedes,ANT,22.0,8.0,1.0,21.0,6.0,5.0,VER,1.0
3,Mercedes,ANT,47.0,6.0,2.0,46.0,5.0,4.0,RUS,2.0
4,Aston Martin,ALO,20.0,15.0,1.0,19.0,13.0,12.0,BOR,1.0


pit_stops table is structured so that each row represents a single pit stop, and every driver-stint combination has exactly one pit stop.

In [ ]:
pit_laps = laps.pick_box_laps(which="in")
pit_laps = pit_laps.dropna(subset=['LapTime'])
pit_laps = pit_laps[['Driver','LapNumber','Position', 'Stint']]
lap_positions = laps[['Driver', 'LapNumber', 'Position', 'Stint']]

In [ ]:


#Finding next pitlap of rival driver

pit_stops = pit_stops.merge(
    pit_laps,
    left_on=['Driver_rival','Stint_rival'],
    right_on=['Driver','Stint'],
    how='left'
)

pit_stops = pit_stops.drop(columns=['Stint_y','Position','Driver_y'])

pit_stops = pit_stops.rename(columns={
    'LapNumber': 'Rival_PitLap',
    'Driver_rival': 'Rival_Driver',
    'PitLap': 'Target_PitLap',
    'Driver_x': 'Target_Driver',
    'Stint_rival': 'Rival_Stint',
    'Stint_x': 'Target_Stint',
    'PositionAtPit': 'Target_PositionAtPitLap',
    'PositionBeforePit': 'Target_PositionBeforePitLap',
    'LapBeforePit': 'Target_LapBeforePit'

})



pit_stops.head(14)


,Team,Target_Driver,Target_PitLap,Target_PositionAtPitLap,Target_Stint,Target_LapBeforePit,Target_PositionBeforePitLap,Rival_Position,Rival_Driver,Rival_Stint,Rival_PitLap
0,Red Bull Racing,VER,37.0,2.0,1.0,36.0,2.0,1.0,NOR,2.0,NaN
1,Alpine,GAS,33.0,18.0,1.0,32.0,17.0,16.0,STR,2.0,43.0
2,Mercedes,ANT,22.0,8.0,1.0,21.0,6.0,5.0,VER,1.0,37.0
3,Mercedes,ANT,47.0,6.0,2.0,46.0,5.0,4.0,RUS,2.0,48.0
4,Aston Martin,ALO,20.0,15.0,1.0,19.0,13.0,12.0,BOR,1.0,37.0
5,Aston Martin,ALO,34.0,16.0,2.0,33.0,15.0,14.0,OCO,2.0,NaN
6,Ferrari,LEC,31.0,2.0,1.0,30.0,2.0,1.0,NOR,1.0,34.0
7,Aston Martin,STR,26.0,12.0,1.0,25.0,11.0,10.0,ANT,2.0,47.0
8,Aston Martin,STR,43.0,11.0,2.0,42.0,11.0,10.0,OCO,2.0,NaN
9,Red Bull Racing,TSU,36.0,9.0,1.0,35.0,9.0,8.0,HAM,2.0,47.0


In [ ]:
pit_laps.groupby(['Driver', 'Stint']).size().describe()
#1 pitstop per stint

,0
count,30.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


In [ ]:
pit_stops.query("Rival_Driver == 'LEC'")[[
    'Rival_Stint',
    'Rival_PitLap'
]].head(10)


,Rival_Stint,Rival_PitLap
17,1.0,31.0
28,1.0,31.0
29,2.0,NaN


In [ ]:
driver_out_laps = laps.pick_box_laps(which='out')
driver_out_laps = driver_out_laps[['Driver', 'Position', 'LapNumber', 'Stint']]
driver_out_laps  = driver_out_laps.rename(columns = {'Driver': 'Rival_Driver',
                                                     'LapNumber': 'Rival_OutLap',
                                                     'Position': 'Rival_Position_On_Rival_OutLap',
                                                     'Stint' : 'Rival_Stint_On_Outlap'})
driver_out_laps.head()
#lap_positions = laps[['Driver', 'LapNumber', 'Position', 'Stint']]

,Rival_Driver,Rival_Position_On_Rival_OutLap,Rival_OutLap,Rival_Stint_On_Outlap
37,VER,8.0,38.0,2.0
104,GAS,18.0,34.0,2.0
163,ANT,13.0,23.0,2.0
188,ANT,8.0,48.0,3.0
232,ALO,19.0,21.0,2.0


In [ ]:
#Introduce target driver back
driver_out_laps = driver_out_laps.merge(
    pit_stops[['Target_Driver', 'Rival_Driver', 'Rival_PitLap', 'Target_Stint','Rival_Stint']],
    on = 'Rival_Driver',
    how = 'left'
)

driver_out_laps.head()


,Rival_Driver,Rival_Position_On_Rival_OutLap,Rival_OutLap,Rival_Stint_On_Outlap,Target_Driver,Rival_PitLap,Target_Stint,Rival_Stint
0,VER,8.0,38.0,2.0,ANT,37.0,1.0,1.0
1,VER,8.0,38.0,2.0,SAI,NaN,2.0,2.0
2,GAS,18.0,34.0,2.0,NaN,NaN,NaN,NaN
3,ANT,13.0,23.0,2.0,STR,47.0,1.0,2.0
4,ANT,13.0,23.0,2.0,PIA,47.0,2.0,2.0


In [ ]:
driver_out_laps.count()

,0
Rival_Driver,49
Rival_Position_On_Rival_OutLap,49
Rival_OutLap,49
Rival_Stint_On_Outlap,49
Target_Driver,44
Rival_PitLap,36
Target_Stint,44
Rival_Stint,44


In [ ]:
#position of target when riveal comes out of pits
lap_positions = laps[['Driver', 'LapNumber', 'Position', 'Stint']]

driver_out_laps = driver_out_laps.merge(
    lap_positions,
    left_on = ['Target_Driver', 'Rival_OutLap'],
    right_on = ['Driver', 'LapNumber'],
    how = 'left'
)
driver_out_laps.head()


,Rival_Driver,Rival_Position_On_Rival_OutLap,Rival_OutLap,Rival_Stint_On_Outlap,Target_Driver,Rival_PitLap,Target_Stint,Rival_Stint,Driver,LapNumber,Position,Stint
0,VER,8.0,38.0,2.0,ANT,37.0,1.0,1.0,ANT,38.0,4.0,2.0
1,VER,8.0,38.0,2.0,SAI,NaN,2.0,2.0,SAI,38.0,9.0,2.0
2,GAS,18.0,34.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ANT,13.0,23.0,2.0,STR,47.0,1.0,2.0,STR,23.0,12.0,1.0
4,ANT,13.0,23.0,2.0,PIA,47.0,2.0,2.0,PIA,23.0,7.0,1.0


In [ ]:
#Renaming columns
#Drop Stint as this refers to Target's stint during the Rival_OutLap
driver_out_laps = driver_out_laps.drop(columns= ['Driver','LapNumber'])
driver_out_laps  = driver_out_laps.rename(columns = {'Stint': 'Target_Stint_On_RivalOutLap',
                                                     'Position': 'Target_Position_On_RivalOutlap',
                                                     })

driver_out_laps.head()

,Rival_Driver,Rival_Position_On_Rival_OutLap,Rival_OutLap,Rival_Stint_On_Outlap,Target_Driver,Rival_PitLap,Target_Stint,Rival_Stint,Target_Position_On_RivalOutlap,Target_Stint_On_RivalOutLap
0,VER,8.0,38.0,2.0,ANT,37.0,1.0,1.0,4.0,2.0
1,VER,8.0,38.0,2.0,SAI,NaN,2.0,2.0,9.0,2.0
2,GAS,18.0,34.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
3,ANT,13.0,23.0,2.0,STR,47.0,1.0,2.0,12.0,1.0
4,ANT,13.0,23.0,2.0,PIA,47.0,2.0,2.0,7.0,1.0


In [ ]:
#pit_stops['Outlap_Rival'] = pit_stops['PitLap_rival'] + 1
#pit_stops

In [ ]:

pit_stops = pit_stops.merge(
  driver_out_laps[['Target_Driver','Rival_Driver', 'Rival_Stint', 'Rival_OutLap', 'Rival_Position_On_Rival_OutLap', 'Target_Position_On_RivalOutlap']],
   left_on=['Target_Driver','Target_Stint','Rival_Driver'],
   right_on=['Target_Driver','Rival_Stint', 'Rival_Driver'],   # Same stint as target
   how ='left'
)

pit_stops

,Team,Target_Driver,Target_PitLap,Target_PositionAtPitLap,Target_Stint,Target_LapBeforePit,Target_PositionBeforePitLap,Rival_Position,Rival_Driver,Rival_Stint_x,Rival_PitLap,Rival_Stint_y,Rival_OutLap,Rival_Position_On_Rival_OutLap,Target_Position_On_RivalOutlap
0,Red Bull Racing,VER,37.0,2.0,1.0,36.0,2.0,1.0,NOR,2.0,NaN,NaN,NaN,NaN,NaN
1,Alpine,GAS,33.0,18.0,1.0,32.0,17.0,16.0,STR,2.0,43.0,NaN,NaN,NaN,NaN
2,Mercedes,ANT,22.0,8.0,1.0,21.0,6.0,5.0,VER,1.0,37.0,1.0,38.0,8.0,4.0
3,Mercedes,ANT,47.0,6.0,2.0,46.0,5.0,4.0,RUS,2.0,48.0,2.0,26.0,10.0,8.0
4,Mercedes,ANT,47.0,6.0,2.0,46.0,5.0,4.0,RUS,2.0,48.0,2.0,49.0,5.0,7.0
5,Aston Martin,ALO,20.0,15.0,1.0,19.0,13.0,12.0,BOR,1.0,37.0,1.0,38.0,14.0,NaN
6,Aston Martin,ALO,34.0,16.0,2.0,33.0,15.0,14.0,OCO,2.0,NaN,2.0,28.0,14.0,15.0
7,Ferrari,LEC,31.0,2.0,1.0,30.0,2.0,1.0,NOR,1.0,34.0,1.0,35.0,1.0,3.0
8,Aston Martin,STR,26.0,12.0,1.0,25.0,11.0,10.0,ANT,2.0,47.0,NaN,NaN,NaN,NaN
9,Aston Martin,STR,43.0,11.0,2.0,42.0,11.0,10.0,OCO,2.0,NaN,2.0,28.0,14.0,17.0
